In [ ]:
import pandas as pd

import pickle

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, log_loss, roc_curve, roc_auc_score, auc, RocCurveDisplay
from sklearn.linear_model import LogisticRegression
from scipy.special import logit

from scipy.stats import norm

In [ ]:
%%time
from google.colab import drive

drive.mount('/content/gdrive')


!cp -r "/content/gdrive/My Drive/recsys/HW1_data.csv" "/content/data.csv"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
CPU times: user 246 ms, sys: 30.5 ms, total: 276 ms
Wall time: 29.8 s


In [ ]:
%%time
data = pd.read_csv('data.csv')
print(data.shape)
print(data.head())

(15821472, 17)
                    date_time  zone_id  banner_id            oaid_hash  \
0  2021-09-27 00:01:30.000000        0          0  5664530014561852622   
1  2021-09-26 22:54:49.000000        1          1  5186611064559013950   
2  2021-09-26 23:57:20.000000        2          2  2215519569292448030   
3  2021-09-27 00:04:30.000000        3          3  6262169206735077204   
4  2021-09-27 00:06:21.000000        4          4  4778985830203613115   

   campaign_clicks  os_id  country_id  banner_id0  rate0        g0  \
0                0      0           0        1240  0.067  0.035016   
1                0      0           1           1  0.002  0.054298   
2                3      0           0           2  0.014  0.014096   
3                0      1           1           3  0.012  0.015232   
4                0      1           0           4  0.019  0.051265   

   coeff_sum0  banner_id1  rate1        g1  coeff_sum1  impressions  clicks  
0   -7.268846           0  0.010  0.04951

Отфильтруем те значения banner_id != banner_id0

In [ ]:
data = data[data['banner_id'] == data['banner_id0']]

In [ ]:
my_feature = ['zone_id', 'banner_id', 'os_id', 'country_id']
#my_feature = ['zone_id', 'banner_id', 'campaign_clicks', 'os_id', 'country_id']

#data_with_names = data[my_feature]
#data_with_names = pd.get_dummies(data_with_names, columns=my_feature, sparse = True)
#column_names = data_with_names.columns
#del data_with_names

Отделим последний день, так как на нем нужно сравнивать

In [ ]:
%%time
data_last_day = data[data['date_time'] >= '2021-10-02']
data = data[data['date_time'] < '2021-10-02']



CPU times: user 4.69 s, sys: 738 ms, total: 5.43 s
Wall time: 5.55 s


In [ ]:
y_data = data['clicks']
y_data_last_day = data_last_day['clicks']

data.drop(['clicks'],axis = 1, inplace = True)
data_last_day.drop(['clicks'],axis = 1, inplace = True)

In [ ]:
data.head()

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,impressions
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,0,1,1,3,0.012,0.015232,-3.461357,99,0.006,0.050671,-3.418403,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,0,1,0,4,0.019,0.051265,-4.009026,11464230,6.790,0.032005,-2.828797,1
5,2021-09-27 00:06:50.000000,5,5,2377014068362699676,0,2,2,5,0.004,0.337634,-3.222757,37,0.004,0.338195,-3.221755,1


In [ ]:
def analysis(data: pd.DataFrame):
    for colomn in data.columns:
        uniq = data[colomn].unique()
        colomn_uniq_size = uniq.size

        print(colomn)
        print('size', colomn_uniq_size)
        print('min, max', uniq.min(), uniq.max())
        if colomn_uniq_size < 50:
            print(uniq)

    pass

analysis(data)

date_time
size 518305
min, max 2021-09-01 00:02:49.000000 2021-10-01 23:59:59.000000
zone_id
size 3185
min, max 0 3443
banner_id
size 1558
min, max 0 1632
oaid_hash
size 5027796
min, max 1116910879938 9223371107563590722
campaign_clicks
size 822
min, max 0 829
os_id
size 11
min, max 0 10
[ 0  1  2  3  4  5  6  7  8  9 10]
country_id
size 17
min, max 0 16
[ 1  0  2  3  4  5  6  8  9 10  7 11 13 12 14 15 16]
banner_id0
size 1558
min, max 0 1632
rate0
size 57
min, max 0.0 0.1
g0
size 11783534
min, max -0.0176373981227117 691.088787242959
coeff_sum0
size 4189282
min, max -8.58897042950208 0.0
banner_id1
size 2678438
min, max 0 14623602
rate1
size 825
min, max nan nan
g1
size 11566529
min, max nan nan
coeff_sum1
size 4570436
min, max nan nan
impressions
size 1
min, max 1 1
[1]


Соберем отдельно данные по послднему дню, для pi стратегии, для baner_id0 и для baner_id1. А так же обучающую выборку на всех данных без последнего дня

In [ ]:


pi_0_data_last_day = data_last_day[['coeff_sum0', 'g0', 'coeff_sum1', 'g1']]

pi_data_last_day = data_last_day[['coeff_sum0', 'g0', 'coeff_sum1', 'g1']]

data_banerid = data[my_feature]

data_last_day_banerid0 = data_last_day[my_feature]
data_last_day_banerid0['banner_id'] = data_last_day['banner_id0']

data_last_day_banerid1 = data_last_day[my_feature]
data_last_day_banerid1['banner_id'] = data_last_day['banner_id1']


<ipython-input-10-a09c78926d5c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_last_day_banerid0['banner_id'] = data_last_day['banner_id0']
<ipython-input-10-a09c78926d5c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_last_day_banerid1['banner_id'] = data_last_day['banner_id1']


Из-за того что baner_id0 содержит слишком много различных значений, которые не будут использоваться при обучении модели, то оставим только те baner_id которые состоят в обучающей выборке, всего их (1600 и значения они принимают до 1632). Так же для того что во всех данных совпали столбцы, объединим все данные в один большой dataframe, сделаем OHE и разъеденим.

In [ ]:
%%time

data_banerid.insert(0, 'type', 'data_banerid')

data_last_day_banerid0.insert(0, 'type', 'data_last_day_banerid0')

data_last_day_banerid1.insert(0, 'type', 'data_last_day_banerid1')

all_data = pd.concat([data_banerid, data_last_day_banerid0, data_last_day_banerid1])




CPU times: user 262 ms, sys: 325 ms, total: 587 ms
Wall time: 584 ms


In [ ]:


analysis(all_data)

type
size 3
min, max data_banerid data_last_day_banerid1
['data_banerid' 'data_last_day_banerid0' 'data_last_day_banerid1']
zone_id
size 3302
min, max 0 3443
banner_id
size 464040
min, max 0 14623601
os_id
size 11
min, max 0 10
[ 0  1  2  3  4  5  6  7  8  9 10]
country_id
size 17
min, max 0 16
[ 1  0  2  3  4  5  6  8  9 10  7 11 13 12 14 15 16]


Из-за того что baner_id1 содержит слишком много

In [ ]:
all_data['banner_id'] = all_data['banner_id']. where(all_data['banner_id']<2000, other=-1)

print(all_data.head(7))
analysis(all_data)

           type  zone_id  banner_id  os_id  country_id
1  data_banerid        1          1      0           1
2  data_banerid        2          2      0           0
3  data_banerid        3          3      1           1
4  data_banerid        4          4      1           0
5  data_banerid        5          5      2           2
6  data_banerid        6          6      3           0
7  data_banerid        7          7      4           1
type
size 3
min, max data_banerid data_last_day_banerid1
['data_banerid' 'data_last_day_banerid0' 'data_last_day_banerid1']
zone_id
size 3302
min, max 0 3443
banner_id
size 1608
min, max -1 1632
os_id
size 11
min, max 0 10
[ 0  1  2  3  4  5  6  7  8  9 10]
country_id
size 17
min, max 0 16
[ 1  0  2  3  4  5  6  8  9 10  7 11 13 12 14 15 16]


In [ ]:
#all_data['banner_id'] = all_data['banner_id']. where(all_data['banner_id']>5000, other=-1)
#all_data.where(all_data.apply(lambda x: x.map(x.value_counts())) >= 30, -1, inplace=True)

all_data = pd.get_dummies(all_data, columns=my_feature, sparse = True)


In [ ]:
%%time

data_banerid, data_last_day_banerid0,


data_banerid = all_data.iloc[0:data_banerid.shape[0]]#all_data[all_data['type'] == 'data_banerid']
data_banerid.drop(['type'],axis = 1, inplace = True)

data_last_day_banerid0 = all_data.iloc[data_banerid.shape[0]:data_banerid.shape[0] + data_last_day_banerid0.shape[0]]#all_data[all_data['type'] == 'data_last_day_banerid0']
data_last_day_banerid0.drop(['type'],axis = 1, inplace = True)

data_last_day_banerid1 = all_data.iloc[data_banerid.shape[0] + data_last_day_banerid0.shape[0]:data_banerid.shape[0] + data_last_day_banerid0.shape[0] + data_last_day_banerid1.shape[0]]#all_data[all_data['type'] == 'data_last_day_banerid1']
data_last_day_banerid1.drop(['type'],axis = 1, inplace = True)

del all_data

<timed exec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 1.71 s, sys: 70.3 ms, total: 1.78 s
Wall time: 1.78 s


<timed exec>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
print(type(data_banerid))
print(type(data))
print(data_banerid.head(2))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   zone_id_0  zone_id_1  zone_id_2  zone_id_3  zone_id_4  zone_id_5  \
1          0          1          0          0          0          0   
2          0          0          1          0          0          0   

   zone_id_6  zone_id_7  zone_id_8  zone_id_9  ...  country_id_7  \
1          0          0          0          0  ...             0   
2          0          0          0          0  ...             0   

   country_id_8  country_id_9  country_id_10  country_id_11  country_id_12  \
1             0             0              0              0              0   
2             0             0              0              0              0   

   country_id_13  country_id_14  country_id_15  country_id_16  
1              0              0              0              0  
2              0              0              0              0  

[2 rows x 4938 columns]


In [ ]:
%%time
mdl= LogisticRegression(solver='liblinear', C = 1.0)
mdl.fit(data_banerid, y_data)

CPU times: user 4min 6s, sys: 39 s, total: 4min 45s
Wall time: 5min 4s


LogisticRegression(solver='liblinear')

In [ ]:
y_data_last_day_banerid0 = mdl.predict_proba(data_last_day_banerid0)

y_data_last_day_banerid1 = mdl.predict_proba(data_last_day_banerid1)

In [ ]:
print('banerid0 roc auc score: ', roc_auc_score(y_data_last_day, y_data_last_day_banerid0[:, 1]))
print('banerid0 log loss score: ', log_loss(y_data_last_day, y_data_last_day_banerid0))


banerid0 roc auc score:  0.7921276393941702
banerid0 log loss score:  0.1337988121811708


Модель обучилась довольно хорошо.

In [ ]:
pi_data_last_day['coeff_sum0'] = logit(y_data_last_day_banerid0[:, 1])
pi_data_last_day['coeff_sum1'] = logit(y_data_last_day_banerid1[:, 1])

<ipython-input-20-0d8b4db53a1a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pi_data_last_day['coeff_sum0'] = logit(y_data_last_day_banerid0[:, 1])
<ipython-input-20-0d8b4db53a1a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pi_data_last_day['coeff_sum1'] = logit(y_data_last_day_banerid1[:, 1])


\xi_0, \xi_1 - нормально распределеные величиы с параметрами \mu_1, \mu_0, sigma_0, sigma_1

P(\xi_0 < \xi_1) = P(\xi_1 - \xi_0 > 0)
Разница двух нормально распределеных величин это нормально распределеное величина, где \mu = \mu_1 - \mu_0 и \sigma^2 = sigma_0^2 + \sigma_1^2


In [ ]:
#pi_0_data_last_day = data_last_day['coeff_sum0', 'g0', 'coeff_sum1', 'g1']

pi0 = norm.cdf((pi_0_data_last_day['coeff_sum0'] - pi_0_data_last_day['coeff_sum1']) / (pi_0_data_last_day['g0']**2 + pi_0_data_last_day['g1']**2)**0.5)

pi = norm.cdf((pi_data_last_day['coeff_sum0'] - pi_data_last_day['coeff_sum1']) / (pi_data_last_day['g0']**2 + pi_data_last_day['g1']**2)**0.5)

In [ ]:
np.minimum(pi / pi0, 10)

array([ 1.        ,  0.99999845, 10.        , ..., 10.        ,
        1.51965228,  1.01437173])

In [ ]:
np.mean(y_data_last_day * np.minimum(pi / (pi0 ), 10))

<ipython-input-25-13b1052df5ed>:1: RuntimeWarning: divide by zero encountered in divide
  np.mean(y_data_last_day * np.minimum(pi / (pi0 ), 10))
<ipython-input-25-13b1052df5ed>:1: RuntimeWarning: overflow encountered in divide
  np.mean(y_data_last_day * np.minimum(pi / (pi0 ), 10))
<ipython-input-25-13b1052df5ed>:1: RuntimeWarning: invalid value encountered in divide
  np.mean(y_data_last_day * np.minimum(pi / (pi0 ), 10))


0.07601045688155975

In [ ]:
print('CIPS :', np.mean(y_data_last_day * np.minimum(pi / (pi0 + 1e-9), 10)))

CIPS : 0.06604802229427255
